In [9]:
import json
import random

def select_random_cities(input_file, output_file, n):
    # Charger les données du fichier cities.json
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Sélectionner aléatoirement n villes dans la liste des villes
    cities = data["cities"]
    selected_cities = random.sample(cities, min(n, len(cities)))  # Prend n villes si possible, sinon toutes

    # Construire la structure de données pour waypoints_data.json
    waypoints = [
        {
            "name": city["label"],
            "coordinates": [float(city["longitude"]), float(city["latitude"])],
            "weight": random.randint(1, 3)  # Poids aléatoire entre 1 et 10
        }
        for city in selected_cities
    ]

    # Ajouter le dépôt (par exemple, Paris) en début de la liste
    waypoints.insert(0, {
        "name": "Depot Paris",
        "coordinates": [2.2472902, 48.8621791],
        "weight": 0
    })

    # Enregistrer les données dans le fichier waypoints_data.json
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(waypoints, f, ensure_ascii=False, indent=2)

# Utilisation de la fonction
select_random_cities("cities.json", "waypoints_data.json", 17)


In [10]:
import numpy as np
from typing import List, Tuple, Dict
import random
from dataclasses import dataclass
import matplotlib.pyplot as plt
import json

@dataclass
class Client:
    def __init__(self, id, name, latitude, longitude, weight):
        self.id = id
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.weight = weight

    def __repr__(self):
        return f"Client({self.id}, '{self.name}', {self.latitude}, {self.longitude}, {self.weight})"

@dataclass
class Vehicle:
    capacite: float = 10.0  # Capacité standard du véhicule en kg

class VRPSolution:
    def __init__(self, routes: List[List[int]], fitness: float = float('inf')):
        self.routes = routes
        self.fitness = fitness

class VRPFrance:
    def __init__(self, 
                 clients: List[Client],
                 depot: Client,
                 vehicle_capacity: float,
                 population_size: int = 1000,
                 generations: int = 100,
                 mutation_rate: float = 0.2,
                 crossover_rate: float = 0.8):
        
        self.clients = clients
        self.depot = depot
        self.vehicle_capacity = vehicle_capacity
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        
        # Matrice des distances
        self.distances = self._calculate_distance_matrix()
        
    def _haversine_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """Calcule la distance en km entre deux points GPS"""
        R = 6371  # Rayon de la Terre en km
        
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(a))
        
        return R * c
        
    def _calculate_distance_matrix(self) -> np.ndarray:
        """Calcule la matrice des distances entre tous les points"""
        n = len(self.clients) + 1  # +1 pour le dépôt
        matrix = np.zeros((n, n))
        
        # Distances depuis/vers le dépôt
        for i, client in enumerate(self.clients):
            matrix[0, i+1] = matrix[i+1, 0] = self._haversine_distance(
                self.depot.latitude, self.depot.longitude,
                client.latitude, client.longitude
            )
        
        # Distances entre clients
        for i, client1 in enumerate(self.clients):
            for j, client2 in enumerate(self.clients):
                if i != j:
                    matrix[i+1, j+1] = self._haversine_distance(
                        client1.latitude, client1.longitude,
                        client2.latitude, client2.longitude
                    )
        
        return matrix
    
    def _create_initial_solution(self) -> VRPSolution:
        """Crée une solution initiale valide avec plusieurs camions"""
        unassigned = list(range(1, len(self.clients) + 1))
        random.shuffle(unassigned)
        
        routes = []
        current_route = []
        current_capacity = 0
        
        # Création des routes
        while unassigned:
            client_id = unassigned[0]
            client_demand = self.clients[client_id-1]
            
            # Si le camion a encore de la capacité, on l'ajoute à la route
            if current_capacity + client_demand.weight <= self.vehicle_capacity:
                current_route.append(client_id)
                current_capacity += client_demand.weight
                unassigned.pop(0)
            else:
                # Si le camion est plein, on crée une nouvelle route (nouveau camion)
                routes.append(current_route)
                current_route = [client_id]  # Nouveau camion
                current_capacity = client_demand.weight
                unassigned.pop(0)
        
        if current_route:
            routes.append(current_route)  # Ajoute la dernière route
        
        return VRPSolution(routes)

    def _calculate_route_time(self, route: List[int]) -> float:
        """Calcule le temps de trajet d'une route en heures (vitesse moyenne 90 km/h)"""
        if not route:
            return 0
            
        distance = self.distances[0, route[0]]  # Dépôt au premier client
        for i in range(len(route)-1):
            distance += self.distances[route[i], route[i+1]]
        distance += self.distances[route[-1], 0]  # Dernier client au dépôt
        
        return distance / 90  # Vitesse moyenne de 90 km/h
    
    def _calculate_fitness(self, solution: VRPSolution) -> float:
        """Calcule la fonction objectif (distance totale)"""
        total_distance = 0
        
        for route in solution.routes:
            if not route:
                continue
            
            # Distance depuis le dépôt
            total_distance += self.distances[0, route[0]]
            
            # Distance entre les clients
            for i in range(len(route)-1):
                total_distance += self.distances[route[i], route[i+1]]
                
            # Retour au dépôt
            total_distance += self.distances[route[-1], 0]
            
        return total_distance
    
    def _crossover(self, parent1: VRPSolution, parent2: VRPSolution) -> VRPSolution:
        """Opérateur de croisement adapté au VRP"""
        if random.random() > self.crossover_rate:
            return parent1
            
        # Sélectionne une route aléatoire de parent1
        route_idx = random.randint(0, len(parent1.routes)-1)
        selected_route = parent1.routes[route_idx]
        
        # Crée une nouvelle solution en gardant la route sélectionnée
        new_routes = [r.copy() for r in parent1.routes]
        
        # Supprime les clients qui sont dans la route sélectionnée des autres routes
        selected_clients = set(selected_route)
        for i, route in enumerate(new_routes):
            if i != route_idx:
                new_routes[i] = [c for c in route if c not in selected_clients]
                
        return VRPSolution(new_routes)
    
    def _mutation(self, solution: VRPSolution) -> VRPSolution:
        """Opérateur de mutation"""
        if random.random() > self.mutation_rate:
            return solution
            
        new_routes = [r.copy() for r in solution.routes]
        
        mutation_type = random.random()
        
        if mutation_type < 0.33 and len(new_routes) >= 2:
            # Échange de clients entre routes
            route1_idx, route2_idx = random.sample(range(len(new_routes)), 2)
            if new_routes[route1_idx] and new_routes[route2_idx]:
                pos1 = random.randint(0, len(new_routes[route1_idx])-1)
                pos2 = random.randint(0, len(new_routes[route2_idx])-1)
                new_routes[route1_idx][pos1], new_routes[route2_idx][pos2] = \
                    new_routes[route2_idx][pos2], new_routes[route1_idx][pos1]
        
        elif mutation_type < 0.66:
            # Inversion d'une sous-séquence dans une route
            if new_routes:
                route_idx = random.randint(0, len(new_routes)-1)
                if len(new_routes[route_idx]) > 2:
                    start = random.randint(0, len(new_routes[route_idx])-2)
                    end = random.randint(start+1, len(new_routes[route_idx])-1)
                    new_routes[route_idx][start:end+1] = reversed(new_routes[route_idx][start:end+1])
        
        else:
            # Déplacement d'un client vers une autre position
            if new_routes:
                route_idx = random.randint(0, len(new_routes)-1)
                if len(new_routes[route_idx]) > 1:
                    old_pos = random.randint(0, len(new_routes[route_idx])-1)
                    new_pos = random.randint(0, len(new_routes[route_idx])-1)
                    client = new_routes[route_idx].pop(old_pos)
                    new_routes[route_idx].insert(new_pos, client)
                
        return VRPSolution(new_routes)
    
    def solve(self) -> Tuple[VRPSolution, List[float]]:
        """Résout le VRP avec l'algorithme génétique"""
        population = [self._create_initial_solution() 
                     for _ in range(self.population_size)]
        
        best_fitness_history = []
        best_solution = None
        best_fitness = float('inf')
        generations_without_improvement = 0
        
        for generation in range(self.generations):
            # Évaluation de la fitness
            for solution in population:
                solution.fitness = self._calculate_fitness(solution)
                if solution.fitness < best_fitness:
                    best_fitness = solution.fitness
                    best_solution = VRPSolution(
                        [r.copy() for r in solution.routes], 
                        solution.fitness
                    )
                    generations_without_improvement = 0
                else:
                    generations_without_improvement += 1
            
            best_fitness_history.append(best_fitness)
            
            # Arrêt si pas d'amélioration depuis longtemps
            if generations_without_improvement > 200:
                break
            
            # Sélection et reproduction
            new_population = []
            population.sort(key=lambda x: x.fitness)
            
            # Élitisme: garde les 2 meilleures solutions
            new_population.extend([population[0], population[1]])
            
            # Crée de nouvelles solutions
            while len(new_population) < self.population_size:
                # Sélection par tournoi
                parent1 = min(random.sample(population, 3), key=lambda x: x.fitness)
                parent2 = min(random.sample(population, 3), key=lambda x: x.fitness)
                
                # Croisement et mutation
                child = self._crossover(parent1, parent2)
                child = self._mutation(child)
                new_population.append(child)
            
            population = new_population
            
        return best_solution, best_fitness_history

    def format_solution(self, solution: VRPSolution) -> List[dict]:
        """Formate la solution pour l'affichage"""
        formatted_routes = []
        
        for i, route in enumerate(solution.routes, 1):
            route_clients = [self.clients[client_id-1] for client_id in route]
            route_distance = self._calculate_fitness(VRPSolution([route]))
            route_time = self._calculate_route_time(route)
            
            formatted_routes.append({
                'vehicle': i,
                'route': f"{self.depot.name} -> " + 
                        " -> ".join(client.name for client in route_clients) +
                        f" -> {self.depot.name}",
                'distance': round(route_distance, 1),
                'time': round(route_time, 1)
            })
        
        return formatted_routes
    


def instance_utilisation(json_file):
    # Chargement des données JSON
    with open(json_file, 'r') as file:
        data = json.load(file)
        
    # Extraction du dépôt (premier élément)
    depot_data = data[0]
    depot = Client(0, depot_data["name"], depot_data["coordinates"][1], depot_data["coordinates"][0], depot_data["weight"])
    
    # Création des clients (reste des éléments)
    clients = []
    for idx, entry in enumerate(data[1:], start=1):
        name = entry["name"]
        latitude, longitude = entry["coordinates"][1], entry["coordinates"][0]  # Inversion car format [lon, lat]
        weight = entry["weight"]
        client = Client(idx, name, latitude, longitude, weight)
        clients.append(client)
    
    # Création et résolution du VRP
    vrp = VRPFrance(
        clients=clients,
        depot=depot,
        vehicle_capacity=20,
        population_size=100,
        generations=1000,
        mutation_rate=0.2,
        crossover_rate=0.8
    )
    
    best_solution, _ = vrp.solve()
    routes = vrp.format_solution(best_solution)
   
    # Enregistrement des routes dans le fichier route.json
    with open("route_gene.json", "w", encoding="utf-8") as outfile:
        routes_data = []
        for route in routes:
            # Extraction des coordonnées des villes dans l'ordre de visite
            coordinates = [[depot.longitude, depot.latitude]]  # Ajouter le dépôt comme point de départ
            times = []  # Stockage des temps de trajet entre chaque étape
            total_time = 0.0  # Temps total initialisé à 0
            last_client = depot  # Point de départ au dépôt

            for client_name in route['route'].split(" -> ")[1:-1]:  # Exclure le dépôt de début et de fin
                # Trouver le client par son nom
                client = next(c for c in clients if c.name == client_name)
                coordinates.append([client.longitude, client.latitude])
                
                # Calcul du temps entre la dernière position et le client actuel
                segment_distance = vrp._haversine_distance(last_client.latitude, last_client.longitude, client.latitude, client.longitude)
                segment_time = segment_distance / 90.0  # Vitesse moyenne de 90 km/h
                total_time += segment_time
                times.append(round(segment_time, 2))
                
                last_client = client  # Mise à jour pour la prochaine étape

            # Ajouter le retour au dépôt
            coordinates.append([depot.longitude, depot.latitude])
            segment_distance = vrp._haversine_distance(last_client.latitude, last_client.longitude, depot.latitude, depot.longitude)
            segment_time = segment_distance / 90.0  # Vitesse moyenne de 90 km/h
            total_time += segment_time
            times.append(round(segment_time, 2))

            # Construction du dictionnaire de route pour chaque camion
            routes_data.append({
                "vehicle": route['vehicle'],
                "order": route['route'],
                "coordinates": coordinates,
                "segment_times": times,
                "total_time": round(total_time, 2)
            })
        
        # Sauvegarde des données dans le fichier JSON
        json.dump(routes_data, outfile, ensure_ascii=False, indent=2)



    # Affichage des résultats
    for route in routes:
        print(f"Camion {route['vehicle']}: {route['route']}")
        print(f"Distance: {route['distance']} km")
        print(f"Durée: {route['time']}h\n")

if __name__ == "__main__":
    instance_utilisation("waypoints_data.json")

Camion 1: Depot Paris -> pissy poville -> st remy en bouzemont st genest -> la tombe -> hericourt -> st mard de vaux -> rogliano -> fontaine sous preaux -> arques la bataille -> pihem -> herin -> Depot Paris
Distance: 2958.7 km
Durée: 32.9h

Camion 2: Depot Paris -> ladon -> sainville -> auzas -> maisons -> chaux des crotenay -> trevol -> vaux sur lunain -> Depot Paris
Distance: 1853.7 km
Durée: 20.6h



In [11]:
import random
from typing import List, Tuple
import json
from copy import deepcopy
import numpy as np
from typing import List, Tuple, Dict
import random
from dataclasses import dataclass
import matplotlib.pyplot as plt
import json
import time
from math import radians, sin, cos, sqrt, atan2

class Client:
    def __init__(self, id, name, latitude, longitude, weight):
        self.id = id
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.weight = weight

class Vehicle:
    def __init__(self, capacity):
        self.capacity = capacity

class VRPSolution:
    def __init__(self, routes):
        self.routes = routes
    
    def __str__(self):
        return str([[client.name for client in route] for route in self.routes])

class TabuVRPFrance:
    def __init__(self, 
                 clients: List[Client], 
                 depot: Client, 
                 vehicle_capacity: float,
                 num_vehicles: int,
                 tabu_tenure: int = 15,
                 max_iterations: int = 100,
                 max_no_improvement: int = 30):
        self.clients = clients
        self.depot = depot
        self.vehicle_capacity = vehicle_capacity
        self.num_vehicles = num_vehicles
        self.tabu_tenure = tabu_tenure
        self.max_iterations = max_iterations
        self.max_no_improvement = max_no_improvement
        self.tabu_list = []

    def haversine_distance(self, lat1, lon1, lat2, lon2) -> float:
        """Calcule la distance de Haversine entre deux points donnés en kilomètres."""
        R = 6371.0  # Rayon moyen de la Terre en kilomètres
        dlat = radians(lat2 - lat1)
        dlon = radians(lon2 - lon1)
        a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        return distance

    def calculate_distance(self, client1: Client, client2: Client) -> float:
        """Calcule la distance de Haversine entre deux clients."""
        return self.haversine_distance(client1.latitude, client1.longitude, client2.latitude, client2.longitude)

    def calculate_route_load(self, route: List[Client]) -> float:
        """Calcule la charge totale d'une route"""
        return sum(client.weight for client in route)

    def calculate_route_cost(self, route: List[Client]) -> float:
        """Calcule le coût total d'une route"""
        if not route:
            return 0
            
        cost = self.calculate_distance(self.depot, route[0])
        for i in range(len(route) - 1):
            cost += self.calculate_distance(route[i], route[i + 1])
        cost += self.calculate_distance(route[-1], self.depot)
        return cost

    def calculate_solution_cost(self, solution: VRPSolution) -> float:
        """Calcule le coût total d'une solution"""
        return sum(self.calculate_route_cost(route) for route in solution.routes)

    def is_route_feasible(self, route: List[Client]) -> bool:
        """Vérifie si une route respecte la contrainte de capacité"""
        return self.calculate_route_load(route) <= self.vehicle_capacity

    def create_initial_solution(self) -> VRPSolution:
        """Crée une solution initiale en répartissant les clients entre plusieurs camions."""
        available_clients = self.clients.copy()
        routes = [[] for _ in range(self.num_vehicles)]  # Une route par camion
        
        for i in range(self.num_vehicles):
            current_route = []
            current_load = 0
            
            while available_clients and len(current_route) < len(self.clients) // self.num_vehicles:
                best_client = None
                best_distance = float('inf')
                
                for client in available_clients:
                    if current_load + client.weight <= self.vehicle_capacity:
                        last_point = self.depot if not current_route else current_route[-1]
                        distance = self.calculate_distance(last_point, client)
                        
                        if distance < best_distance:
                            best_distance = distance
                            best_client = client
                
                if best_client is None:
                    break
                    
                current_route.append(best_client)
                current_load += best_client.weight
                available_clients.remove(best_client)
                
            routes[i] = current_route  # Associe la route au camion i
            
        # Redistribue les clients restants si des camions sont encore vides
        for client in available_clients:
            for route in routes:
                if self.calculate_route_load(route) + client.weight <= self.vehicle_capacity:
                    route.append(client)
                    break
                
        return VRPSolution(routes)
    

    def generate_neighbors(self, solution: VRPSolution) -> List[Tuple[VRPSolution, Tuple]]:
        """Génère des solutions voisines en utilisant des échanges entre camions."""
        neighbors = []
        
        for i in range(len(solution.routes)):
            for j in range(len(solution.routes)):
                for pos1 in range(len(solution.routes[i])):
                    max_pos2 = len(solution.routes[j]) if i != j else pos1
                    for pos2 in range(max_pos2):
                        if i == j and pos1 == pos2:
                            continue
                            
                        new_solution = VRPSolution([route[:] for route in solution.routes])
                        
                        # Échange les clients entre les routes des camions
                        new_solution.routes[i][pos1], new_solution.routes[j][pos2] = \
                            new_solution.routes[j][pos2], new_solution.routes[i][pos1]
                        
                        # Vérifie la faisabilité pour les deux routes modifiées
                        if (self.is_route_feasible(new_solution.routes[i]) and 
                            self.is_route_feasible(new_solution.routes[j])):
                            move = (i, j, pos1, pos2)
                            neighbors.append((new_solution, move))
        
        return neighbors

    def solve_with_tabu(self) -> Tuple[VRPSolution, List[float]]:
        """Résout le VRP avec une recherche tabou"""
        current_solution = self.create_initial_solution()
        best_solution = deepcopy(current_solution)
        best_cost = self.calculate_solution_cost(best_solution)
        
        costs_history = [best_cost]
        iterations_without_improvement = 0
        
        for _ in range(self.max_iterations):
            # Génère et évalue les voisins
            neighbors = self.generate_neighbors(current_solution)
            if not neighbors:
                break
                
            # Trouve le meilleur voisin non tabou
            best_neighbor = None
            best_neighbor_cost = float('inf')
            best_move = None
            
            for neighbor, move in neighbors:
                if move not in self.tabu_list:
                    cost = self.calculate_solution_cost(neighbor)
                    if cost < best_neighbor_cost:
                        best_neighbor = neighbor
                        best_neighbor_cost = cost
                        best_move = move
            
            if best_neighbor is None:
                break
                
            # Met à jour la solution courante
            current_solution = best_neighbor
            
            # Met à jour la meilleure solution si nécessaire
            if best_neighbor_cost < best_cost:
                best_solution = deepcopy(best_neighbor)
                best_cost = best_neighbor_cost
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
            
            # Met à jour la liste tabou
            self.tabu_list.append(best_move)
            if len(self.tabu_list) > self.tabu_tenure:
                self.tabu_list.pop(0)
            
            costs_history.append(best_cost)
            
            # Critère d'arrêt
            if iterations_without_improvement >= self.max_no_improvement:
                break
        
        return best_solution, costs_history

# Exemple d'utilisation
if __name__ == "__main__":
    json_file = "waypoints_data.json"  
    with open(json_file, 'r') as file:
        data = json.load(file)
        
    # Création des données de test
    depot_data = data[0]
    depot = Client(0, depot_data["name"], depot_data["coordinates"][1], depot_data["coordinates"][0], depot_data["weight"])
    clients = []
    for idx, entry in enumerate(data[1:], start=1):
        name = entry["name"]
        latitude, longitude = entry["coordinates"][1], entry["coordinates"][0]  # Inversion car format [lon, lat]
        weight = entry["weight"]
        client = Client(idx, name, latitude, longitude, weight)
        clients.append(client)
    
    # Création et résolution du problème
    vrp = TabuVRPFrance(
        clients=clients,
        depot=depot,
        num_vehicles=6,
        vehicle_capacity=20
    )
    
    start_time = time.time()
    best_solution, fitness_history = vrp.solve_with_tabu()
    runtime = time.time() - start_time
    
    best_solution_cost = vrp.calculate_solution_cost(best_solution)
    average_solution_cost = np.mean(fitness_history)
    
    # Export du meilleur itinéraire dans un fichier JSON
with open("route_tabou.json", "w", encoding="utf-8") as outfile:
    routes_data = []
    for idx, route in enumerate(best_solution.routes):
        # Extraction des coordonnées des villes dans l'ordre de visite
        coordinates = [[depot.longitude, depot.latitude]]  # Ajouter le dépôt comme point de départ
        times = []  # Stockage des temps de trajet entre chaque étape
        total_time = 0.0  # Temps total initialisé à 0
        last_client = depot  # Point de départ au dépôt

        for client in route:
            coordinates.append([client.longitude, client.latitude])
            
            # Calcul du temps entre la dernière position et le client actuel
            segment_distance = vrp.calculate_distance(last_client, client)
            segment_time = segment_distance / 90.0  # Vitesse moyenne de 90 km/h
            total_time += segment_time
            times.append(round(segment_time, 2))
            
            last_client = client  # Mise à jour pour la prochaine étape

        # Ajouter le retour au dépôt
        coordinates.append([depot.longitude, depot.latitude])
        segment_distance = vrp.calculate_distance(last_client, depot)
        segment_time = segment_distance / 90.0  # Vitesse moyenne de 90 km/h
        total_time += segment_time
        times.append(round(segment_time, 2))

        # Construction du dictionnaire de route pour chaque camion
        routes_data.append({
            "vehicle": idx + 1,
            "order": " -> ".join([depot.name] + [client.name for client in route] + [depot.name]),
            "coordinates": coordinates,
            "segment_times": times,
            "total_time": round(total_time, 2)
        })

    # Sauvegarde des données dans le fichier JSON
    json.dump(routes_data, outfile, ensure_ascii=False, indent=2)



In [37]:
import json
import math
import pulp
from itertools import product

# Classe Client pour gérer les points de livraison et le dépôt
class Client:
    def __init__(self, id, name, latitude, longitude, weight):
        self.id = id
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.weight = weight

# Fonction pour charger les waypoints depuis un fichier JSON
def load_waypoints(filename):
    with open(filename, 'r') as file:
        waypoints = json.load(file)
    return waypoints

# Calcul de la distance de Haversine entre deux points
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Rayon moyen de la Terre en km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Calcul de la distance euclidienne (utilisée dans le modèle de résolution)
def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Charger les données
waypoints = load_waypoints('waypoints_data.json')
num_points = len(waypoints)

# Définir les clients et le dépôt
depot_data = waypoints[0]
depot = Client(0, depot_data["name"], depot_data["coordinates"][1], depot_data["coordinates"][0], depot_data["weight"])
clients = [
    Client(idx, wp["name"], wp["coordinates"][1], wp["coordinates"][0], wp["weight"])
    for idx, wp in enumerate(waypoints[1:], start=1)
]

# Calculer la matrice de distances
distance_matrix = [[0] * num_points for _ in range(num_points)]
for i, wp1 in enumerate(waypoints):
    for j, wp2 in enumerate(waypoints):
        if i != j:
            distance_matrix[i][j] = calculate_distance(wp1['coordinates'], wp2['coordinates'])

# Calcul de la capacité maximale de chaque véhicule
vehicle_capacity = 20

# Définir le problème VRP
vrp_problem = pulp.LpProblem("Vehicle_Routing_Problem", pulp.LpMinimize)

# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_points) for j in range(num_points)),
                          cat='Binary')
u = pulp.LpVariable.dicts("u", (i for i in range(num_points)), lowBound=0, cat='Continuous')

# Fonction objectif (minimiser la distance totale)
vrp_problem += pulp.lpSum(distance_matrix[i][j] * x[i, j] for i in range(num_points) for j in range(num_points))

# Contraintes pour garantir que chaque point est visité une fois
for i in range(1, num_points):
    vrp_problem += pulp.lpSum(x[i, j] for j in range(num_points) if i != j) == 1

for j in range(1, num_points):
    vrp_problem += pulp.lpSum(x[i, j] for i in range(num_points) if i != j) == 1

# Contraintes pour les entrées et sorties du dépôt
vrp_problem += pulp.lpSum(x[0, j] for j in range(1, num_points)) == 1
vrp_problem += pulp.lpSum(x[i, 0] for i in range(1, num_points)) == 1

# Contrainte de sous-tours pour assurer que le trajet est continu
for i in range(1, num_points):
    for j in range(1, num_points):
        if i != j:
            vrp_problem += u[i] - u[j] + (num_points - 1) * x[i, j] <= num_points - 2

# Ajout de la contrainte de capacité
for i in range(1, num_points):
    vrp_problem += u[i] <= vehicle_capacity  # Capacité maximale pour chaque véhicule

# Résolution du problème
vrp_problem.solve()

# Extraction de la solution en suivant le chemin optimisé
solution = []
current_node = 0  # Commencer depuis le dépôt
visited = {current_node}
current_weight = 0  # Pour suivre le poids du véhicule actuel

while len(visited) < num_points:
    for j in range(num_points):
        if pulp.value(x[current_node, j]) == 1:
            solution.append((current_node, j))
            visited.add(j)
            current_node = j
            break

# Export JSON du trajet optimal
with open("route_exact.json", "w", encoding="utf-8") as outfile:
    routes_data = []
    current_vehicle = 1
    route = []
    coordinates = [[depot.longitude, depot.latitude]]  # Point de départ au dépôt
    times = []  # Temps entre chaque étape
    total_time = 0.0  # Temps total
    current_weight = 0  # Suivi de la charge actuelle du véhicule
    last_client = depot  # Point de départ

    # Reconstitution de l'ordre des visites
    for i, j in solution:
        client = clients[j - 1] if j != 0 else depot
        if current_weight + client.weight > vehicle_capacity:
            # Si la capacité est dépassée, terminer la route actuelle et commencer une nouvelle
            coordinates.append([depot.longitude, depot.latitude])
            segment_distance = haversine_distance(last_client.latitude, last_client.longitude, depot.latitude, depot.longitude)
            segment_time = segment_distance / 90.0  # Vitesse moyenne de 90 km/h
            total_time += segment_time
            times.append(round(segment_time, 2))

            # Enregistrer la route terminée
            routes_data.append({
                "vehicle": current_vehicle,
                "order": " -> ".join([depot.name] + [client.name for client in route] + [depot.name]),
                "coordinates": coordinates,
                "segment_times": times,
                "total_time": round(total_time, 2)
            })

            # Réinitialiser pour un nouveau véhicule
            current_vehicle += 1
            route = []
            coordinates = [[depot.longitude, depot.latitude]]  # Dépôt ajouté au début
            times = []
            total_time = 0.0
            current_weight = 0
            last_client = depot

        # Enregistrer chaque étape du trajet
        route.append(client)
        coordinates.append([client.longitude, client.latitude])
        current_weight += client.weight

        # Calcul du temps entre la dernière position et le client actuel
        segment_distance = haversine_distance(last_client.latitude, last_client.longitude, client.latitude, client.longitude)
        segment_time = segment_distance / 90.0  # Vitesse moyenne de 90 km/h
        total_time += segment_time
        times.append(round(segment_time, 2))

        last_client = client  # Mise à jour pour la prochaine étape

    # Enregistrer la dernière route si elle n'est pas vide
    if route:
        coordinates.append([depot.longitude, depot.latitude])  # Dépôt ajouté à la fin
        segment_distance = haversine_distance(last_client.latitude, last_client.longitude, depot.latitude, depot.longitude)
        segment_time = segment_distance / 90.0  # Vitesse moyenne de 90 km/h
        total_time += segment_time
        times.append(round(segment_time, 2))
        routes_data.append({
            "vehicle": current_vehicle,
            "order": " -> ".join([depot.name] + [client.name for client in route] + [depot.name]),
            "coordinates": coordinates,
            "segment_times": times,
            "total_time": round(total_time, 2)
        })

    json.dump(routes_data, outfile, ensure_ascii=False, indent=2)


In [13]:
import folium
import json

def creer_map_route(json_file: str, map_output: str = "map.html"):
    # Charger les données de trajets depuis le fichier route.json
    with open(json_file, "r", encoding="utf-8") as file:
        routes_data = json.load(file)
    
    # Extraire les coordonnées du dépôt (première coordonnée de la première route)
    depot_coords = routes_data[0]["coordinates"][0]
    depot_coords = [depot_coords[1], depot_coords[0]]  # Inverser pour [latitude, longitude]

    # Créer une carte centrée sur le dépôt
    map_folium = folium.Map(location=depot_coords, zoom_start=7)

    # Liste de 30 couleurs pour les différents camions
    couleurs_camions = [
        "blue", "green", "orange", "purple", "red", "brown", "pink", "darkblue", "darkgreen", "darkorange",
        "cadetblue", "crimson", "darkviolet", "gold", "indigo", "lightcoral", "lightblue", "lightgreen", "magenta", "maroon",
        "mediumvioletred", "navy", "olive", "purple", "seagreen", "skyblue", "slateblue", "teal", "tomato", "violet"
    ]

    # Tracer chaque route
    for idx, route in enumerate(routes_data):
        coordinates = [[coord[1], coord[0]] for coord in route["coordinates"]]  # Inverser les coordonnées
        
        # Assigner une couleur différente à chaque camion
        couleur_camion = couleurs_camions[idx % len(couleurs_camions)]  # Utiliser un indice cyclique pour les couleurs
        
        # Ajouter le tracé de la route
        folium.PolyLine(
            coordinates,
            color=couleur_camion,
            weight=5,
            opacity=0.7,
            tooltip=f"Camion {route['vehicle']}"
        ).add_to(map_folium)
        
        # Ajouter des marqueurs pour chaque ville
        for i, coord in enumerate(coordinates):
            folium.Marker(
                location=coord,
                icon=folium.Icon(color="green" if i == 0 else "red"),
                popup=f"{'Dépot' if i == 0 else 'Ville'} - Camion {route['vehicle']}"
            ).add_to(map_folium)

    # Sauvegarder la carte dans un fichier HTML
    map_folium.save(map_output)
    print(f"Carte enregistrée sous le nom {map_output}")

# Utiliser la fonction pour générer la carte à partir de route.json
creer_map_route("route.json")

IndexError: list index out of range